In [1]:
import json
#from eval import main
import numpy as np
import nltk
import random
#import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import transformers


In [2]:
#!pip install autokeras
#!pip install --upgrade tensorflow==2.12 protobuf==4.21 tensorboard==2.12 tensorflow-estimator==2.12 keras ==2.12
#!pip show tensorflow-intel 




In [2]:
def label_t(label):
    if label == 'SUPPORTS':
        t = 0
    elif label == 'REFUTES':
        t = 1
    elif label == 'NOT_ENOUGH_INFO':
        t = 2
    elif label == 'DISPUTED':
        t = 3
    else:
        return None
    return t

    

In [3]:
with open('dataset/evidence.json') as json_file:
    evidence = json.load(json_file)

with open('dataset/dev-claims.json') as json_file:
    dev_claim = json.load(json_file)

with open('dataset/dev-claims-baseline.json') as json_file:
    dev_claims_base = json.load(json_file)

with open('dataset/train-claims.json') as json_file:
    train = json.load(json_file)



In [5]:
with open('dataset/test-claims-unlabelled.json') as json_file:
    test = json.load(json_file)

In [6]:
claims_test = []
amount = []
for key in test.keys():
    test[key]['claim_text'] = test[key]['claim_text'].lower()
    sentence = test[key]['claim_text'].split()
    word = [i for i in (sentence) if i not in set(stopwords.words('english'))] 
    #print(word)
    amount += word
    word = ' '.join(word)
    claims_test.append(word)

word_amount = list(set(amount))
#print(len(word_amount))
vectorizer = CountVectorizer(max_features = len(amount))
bow_claim = vectorizer.fit_transform(claims_test).toarray()

In [14]:

w_evi = []
for key in train.keys():
    word = []
    for evi in train[key]['evidences']:
        word.append(evi[9:])
        #print(evi)
        #print(evi[9:])
        #break
    word = ' '.join(word)
    w_evi.append(word)
    #print(word)
    #sentence = train[key]['claim_label'].split()
    #word = [i for i in (sentence) if i not in set(stopwords.words('english'))] 
    #print(word)
    #word_amount += word
    #word = ' '.join(word)
    #responds.append(word)



vectorizer = CountVectorizer()
evidence_train = vectorizer.fit_transform(w_evi).toarray()

#print(evidence_train)





3


In [11]:
responds = []
counter = 0
w_claim = []
word_amount = []
label = []
for key in train.keys():
    
    rep = label_t(train[key]['claim_label'])
    if rep == None:
        print("error response")
        break
    #else:
        #responds.append(rep)
        """
        counter +=1
        if rep in responds.keys():
            responds[rep] +=1
        else:
            responds[rep] =1
        """
    
    train[key]['claim_text'] = train[key]['claim_text'].lower()
    sentence = train[key]['claim_text'].split()
    word = [i for i in (sentence) if i not in set(stopwords.words('english'))] 
    #print(word)
    word_amount += word
    word = ' '.join(word)
    w_claim.append(word)

    sentence = train[key]['claim_label'].split()
    word = [i for i in (sentence) if i not in set(stopwords.words('english'))] 
    #print(word)
    #word_amount += word
    word = ' '.join(word)
    responds.append(word)



word_amount = list(set(word_amount))
#print(len(word_amount))
vectorizer = CountVectorizer(max_features = len(word_amount))
bow_claim = vectorizer.fit_transform(w_claim).toarray()

vectorizer = CountVectorizer()
responds = vectorizer.fit_transform(responds).toarray()


In [7]:
from transformers import BertModel
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("gpt2")


print("Done loading BERT model.")

Done loading BERT model.


In [11]:
claim_text = {}
for key in train.keys():
    train[key]['claim_text'] = train[key]['claim_text'].lower()
    claim_text[key] = tokenizer(train[key]['claim_text'], padding=False, truncation=True)
    print(claim_text[key])
    break


{'input_ids': [1662, 691, 318, 612, 645, 5654, 2370, 326, 763, 17, 318, 257, 36156, 415, 11, 2440, 763, 17, 14587, 1682, 1037, 30020, 1104, 517, 4618, 290, 5044, 1204, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [14]:
pre = np.load('test.npy')


array([['DISPUTED'],
       ['REFUTES'],
       ['SUPPORTS'],
       ...,
       ['NOT_ENOUGH_INFO'],
       ['NOT_ENOUGH_INFO'],
       ['SUPPORTS']], dtype='<U15')

In [3]:

pre = np.load('test.npy').tolist()


with open('test-claims-prediction.json') as json_file:
    format = json.load(json_file)

counter = 0
for key in format.keys():
    format[key]['claim_label'] = pre[counter][0]
    counter +=1

file_path = '1.json'
with open(file_path, 'w') as json_file:
    json.dump(format, json_file)